This notebook included steps for the Segmenting and Clustering Neighborhoods in Toronto

In [4]:
import requests
import lxml.html as lh
import pandas as pd

Extracting the list of postal codes in Canda by scraping a Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

col=[]
i=0
#Extract the column names from the first table row
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    i=0
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    i=0
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Converting the extracted data to pandas data frame

In [6]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

# Remove records with Not Assigned Borough from the dataframe
df=df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n','')

# Replace Not assigned Neighbourhood with the value of Borough
df['Neighbourhood'] = df['Neighbourhood'].replace('Not assigned',df['Borough'])

# Combine rows with the same postal code in one row , having Neighbourhood as comma separated
df['Neighbourhood'] = df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df=df[['Postcode','Borough','Neighbourhood']].drop_duplicates().reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [7]:
df.shape

(103, 3)